## Latent Dirichlet Allocation (LDA) 

We start with simple LDA to generate topics for each citation statement
 

## Preprocessing

to ensure the citations are stripped of html and preprocessed according to a pipeline 

In [31]:
from bs4 import BeautifulSoup
import string
import re

def remove_html(x):
    soup = BeautifulSoup(x, 'html.parser')
    tags = soup.find_all('cite')
    text = soup.get_text()
    text = text.replace('  ', '')
    text = text.replace(' et ', '')
    text = text.replace(' al ', '')
    text = text.replace(' et. ', '')
    text = text.replace(' al. ', '')
    return text

In [32]:
import scispacy
import spacy

nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("merge_entities")

def merge_entities(x):
    doc = nlp(x)
    return " ".join([t.text.replace(" ", "_").lower() for t in doc])


/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point

/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point

/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point

In [33]:
import pandas as pd

citations = pd.read_csv('./example_doc_citations.csv')
citations['text'] = citations['text'].apply(
    lambda x : remove_html(x)
)
citations['ner_merged_text'] = citations['text'].apply(
    lambda x: merge_entities(x)
)

/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [78]:
removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
remove_text = ['al', 'et', 'al.']
authors = [
    'mongeon',
    'hus'
]

preprocessed_documents = []
for doc in nlp.pipe(citations['ner_merged_text']):
    proj_tok = [token.lemma_.lower().replace('-', '_') for token in doc if "_" in token.text or (len(token.text) > 2 and token.text not in remove_text and token.pos_ not in removal and not token.is_stop and token.is_alpha)]
    proj_tok = [
        tok for tok in proj_tok
        if all([author not in tok for author in authors])
    ]
    preprocessed_documents.append(' '.join(proj_tok))

/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [79]:
def get_topics_lda(model, dictionary, num_topics, topk=10):
    topic_terms = []
    for i in range(num_topics):
        topic_words_list = []
        for word_tuple in model.get_topic_terms(i, topk):
            topic_words_list.append(dictionary[word_tuple[0]])
        topic_terms.append(topic_words_list)
    return topic_terms

In [80]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from contextualized_topic_models.evaluation.measures import InvertedRBO

num_topics = [3, 5, 10, 15, 20, 25, 50, 100, 250]

texts = [doc.split() for doc in preprocessed_documents]
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.9)
corpus = [dictionary.doc2bow(text) for text in texts]

model_results = []
for n_components in num_topics:
    print("num topics:", n_components)
    lda = LdaMulticore(corpus, num_topics=n_components, random_state=42, id2word=dictionary, iterations=100, workers = 4, passes=4)
    cm = CoherenceModel(model=lda, dictionary=dictionary, texts=texts, coherence='c_npmi')
    coh_score = cm.get_coherence()
    print("coherence score:", coh_score)
    diversity_score = InvertedRBO(get_topics_lda(lda, dictionary, n_components)).score()
    print("diversity score LDA:", diversity_score)
    model_results.append({
        "coherence": coh_score,
        "diversity": diversity_score,
        "num_topics": n_components,
        "model": lda
    })

num topics: 3
coherence score: -0.007323554607511229
diversity score LDA: 0.4201705206666666
num topics: 5
coherence score: -0.009137490743558383
diversity score LDA: 0.3804983133571429
num topics: 10
coherence score: -0.01273115002176257
diversity score LDA: 0.4448318440453969
num topics: 15
coherence score: -0.018948219275307866
diversity score LDA: 0.5065354569213606
num topics: 20
coherence score: -0.025575206624108608
diversity score LDA: 0.5116107151306015
num topics: 25
coherence score: -0.030237372450631593
diversity score LDA: 0.5565449973271905
num topics: 50
coherence score: -0.0500608996170644
diversity score LDA: 0.6442292893809214
num topics: 100
coherence score: -0.07440194223504268
diversity score LDA: 0.7196760419157215
num topics: 250
coherence score: -0.1267254829363632
diversity score LDA: 0.8695651072063089


## Select Top Topic Model

In [81]:
results_df = pd.DataFrame(model_results)

In [82]:
results_df.sort_values(by='coherence', ascending=False)

,coherence,diversity,num_topics,model
0,-0.007324,0.420171,3,"LdaMulticore<num_terms=1673, num_topics=3, dec..."
1,-0.009137,0.380498,5,"LdaMulticore<num_terms=1673, num_topics=5, dec..."
2,-0.012731,0.444832,10,"LdaMulticore<num_terms=1673, num_topics=10, de..."
3,-0.018948,0.506535,15,"LdaMulticore<num_terms=1673, num_topics=15, de..."
4,-0.025575,0.511611,20,"LdaMulticore<num_terms=1673, num_topics=20, de..."
5,-0.030237,0.556545,25,"LdaMulticore<num_terms=1673, num_topics=25, de..."
6,-0.050061,0.644229,50,"LdaMulticore<num_terms=1673, num_topics=50, de..."
7,-0.074402,0.719676,100,"LdaMulticore<num_terms=1673, num_topics=100, d..."
8,-0.126725,0.869565,250,"LdaMulticore<num_terms=1673, num_topics=250, d..."


In [83]:
results_df.sort_values(by='diversity', ascending=False)

,coherence,diversity,num_topics,model
8,-0.126725,0.869565,250,"LdaMulticore<num_terms=1673, num_topics=250, d..."
7,-0.074402,0.719676,100,"LdaMulticore<num_terms=1673, num_topics=100, d..."
6,-0.050061,0.644229,50,"LdaMulticore<num_terms=1673, num_topics=50, de..."
5,-0.030237,0.556545,25,"LdaMulticore<num_terms=1673, num_topics=25, de..."
4,-0.025575,0.511611,20,"LdaMulticore<num_terms=1673, num_topics=20, de..."
3,-0.018948,0.506535,15,"LdaMulticore<num_terms=1673, num_topics=15, de..."
2,-0.012731,0.444832,10,"LdaMulticore<num_terms=1673, num_topics=10, de..."
0,-0.007324,0.420171,3,"LdaMulticore<num_terms=1673, num_topics=3, dec..."
1,-0.009137,0.380498,5,"LdaMulticore<num_terms=1673, num_topics=5, dec..."


In [128]:
lda_model = results_df.sort_values(by='coherence', ascending=False).iloc[5]['model']

In [140]:
topics = get_topics_lda(lda_model, dictionary, results_df.sort_values(by='coherence', ascending=False).iloc[5]['num_topics'], topk=8)
topics

[['wos',
  'journal',
  'scopus',
  'database',
  'article',
  'search',
  'research',
  'web_of_science'],
 ['scopus',
  'wos',
  'source',
  'language',
  'publication',
  'country',
  'datum',
  'bias'],
 ['search',
  'scopus',
  'web_of_science',
  'academic',
  'study',
  'wos',
  'engine',
  'result'],
 ['database',
  'scopus',
  'analysis',
  'journal',
  'search',
  'coverage',
  'web_of_science',
  'research'],
 ['database',
  'scopus',
  'wos',
  'web_of_science',
  'coverage',
  'journal',
  'search',
  'source'],
 ['study',
  'database',
  'publication',
  'wos',
  'research',
  'journal',
  'article',
  'coverage'],
 ['scopus',
  'research',
  'web_of_science',
  'coverage',
  'google_scholar',
  'wos',
  'identify',
  'trend'],
 ['source',
  'scopus',
  'web_of_science',
  'search',
  'database',
  'wos',
  'research',
  'study'],
 ['wos',
  'scopus',
  'journal',
  'database',
  'web_of_science',
  'publication',
  'index',
  'criterion'],
 ['scopus',
  'study',
  'web_o

In [141]:
topics_without_overlaps = []
for i, topic in enumerate(topics):
    topic_set = set(topic)
    for j, topic in enumerate(topics):
        if i == j:
            continue
        topic_set = topic_set - set(topic)
    topics_without_overlaps.append(list(topic_set))
topics_without_overlaps

[[],
 ['country'],
 ['academic', 'engine'],
 ['analysis'],
 [],
 [],
 ['trend', 'identify', 'google_scholar'],
 [],
 ['index', 'criterion'],
 ['provide'],
 [],
 ['bibliometric_analysis', 'term'],
 [],
 ['compare'],
 ['include'],
 [],
 [],
 ['social_science', 'humanity'],
 ['literature'],
 ['subject', 'publish'],
 ['information', 'base'],
 [],
 ['review'],
 ['wide'],
 []]

## Visualize topics

In [131]:

import pyLDAvis
from pyLDAvis.gensim_models import prepare

lda_display = prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

/Users/domenicrosati/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## Compute Topic Index

In [144]:
import numpy as np
for i, result in results_df.iterrows():
    documents = []
    for document, doi, cite_id in zip(corpus, citations['source_doi'], citations['id']):
        topics = result['model'].get_document_topics(document)
        topic = topics[np.argmax([topic[1] for topic in topics])]
        keywords = [term[0] for term in result['model'].get_topic_terms(topic[0], 10)]
        documents.append({
            "doi": doi,
            "cite_id": cite_id,
            "keywords": [dictionary[word] for word in set(keywords)]
        })
        with open(f'./lda_{result["num_topics"]}_topic_index.json', 'w+') as f:
            json.dump({
                "topics": result["num_topics"],
                "diversity": result["diversity"],
                "coherence": result["coherence"],
                "documents": documents
            }, f)